In [3]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
from tqdm import tqdm

In [4]:
print('CUDA GPU available : {}'.format(torch.cuda.is_available()))
try:
    print('{} GPU(s) is(are) allocated'.format(torch.cuda.device_count()))
except:
    print('GPUs are not allocated. Current runtime is on CPU.')
device = torch.device("cuda")

CUDA GPU available : True
1 GPU(s) is(are) allocated


In [5]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'
train_dir = '/opt/ml/input/data/train'

## Dataset

In [6]:
class TrainDataset(Dataset):
    def __init__(self, train_dir, is_Train=True, transform=None):
        super().__init__()
        
        csv_path = os.path.join(train_dir, 'train.csv')
        csv = pd.read_csv(csv_path)
        self.image_dir = os.path.join(train_dir, 'images')
        self.transform = transform
        self.image_path = []
        path = csv['path']
        
        for p in path:
            images = [os.path.join(*[self.image_dir, p, image]) for image in os.listdir(os.path.join(self.image_dir, p)) if not image[:1] == '.']
            for image in images:
                self.image_path.append(image)
                
        self.comb_dic = {}
        comb = [(m, g, a) for m in ['m', 'i', 'n'] for g in ['male', 'female'] for a in [0, 1, 2]]
        for i, (m, g, a) in enumerate(comb):
            self.comb_dic[(m, g, a)] = i

    def __len__(self):
        return len(self.image_path)

    def __getitem__(self, idx):
        image_name = self.image_path[idx]
        image = Image.open(image_name)
        
        features = image_name.split('/')[-2:]
        mask = features[1][0]
        age = int(features[0].split('_')[-1])
        gender = features[0].split('_')[1]
        
        if age >= 57: # 원래 60
            age = 2
        elif age >= 26: # 원래 30
            age = 1
        else:
            age = 0        

        target = self.comb_dic[(mask, gender, age)]
        
        if self.transform:
            image = self.transform(image)
        
        return image, target

class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [27]:
from torch.utils.data.dataset import random_split
tfms = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])
# tfms = transforms.Compose([
# #     transforms.RandomRotation(10),
#     transforms.CenterCrop((450, 360)),
#     #transforms.Resize((224, 224)), 
#     transforms.ToTensor(),
#     #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
# ])
dataset = TrainDataset(train_dir, transform=tfms)
train_dataset, val_dataset = random_split(dataset, [int(len(dataset)*0.8),int(len(dataset)*0.2)])
# print(len(dataset))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=1, drop_last=False)
val_loader   = DataLoader(dataset=val_dataset, batch_size=64)

## Model

In [6]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b4', num_classes=18).to(device)

Loaded pretrained weights for efficientnet-b4


In [7]:
import torch.optim as optim
lr = 3e-4

class FocalLoss(nn.Module):
    def __init__(self, weight=None,
                 gamma=0.8, reduction='mean'):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            ((1 - prob) ** self.gamma) * log_prob,
            target_tensor,
            weight=self.weight,
            reduction=self.reduction
        )
    
# criterion = nn.CrossEntropyLoss()
criterion = FocalLoss()
optimizer = optim.AdamW(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.2)

In [8]:
from sklearn.metrics import confusion_matrix

def check_f1(loader, length, model, device):
    model.eval()
    y_true = []
    y_predicted = []
    with torch.no_grad():
        prog_bar = tqdm(enumerate(loader), total=int(length/loader.batch_size))
        for i, (x, y) in prog_bar:
            y_true += y.tolist()

            x = x.to(device=device)
            y = y.to(device=device)


            scores = model(x)
            _, predictions = scores.max(1)
            y_predicted += predictions.tolist()

    cm = confusion_matrix(y_true, y_predicted)
    F1 = []
    for c in range(18):
        precision = cm[c][c] / np.sum(cm, axis=0)[c]
        recall = cm[c][c] / np.sum(cm, axis=1)[c]
        F1.append(2 * precision * recall / (precision + recall))
    macro_F1 = np.mean(F1)
    print("Confusion Matrix")
    for row in cm:
        print(row)
    print("Validation F1 score :" , macro_F1)
    model.train()

def check_accuracy(loader, model, device):
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        
        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}') 
    
    model.train()

In [10]:
NUM_EPOCHS = 3

for epoch in range(NUM_EPOCHS):
    
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    total = 0
    total_it = int(len(train_dataset)/train_loader.batch_size)
    prog_bar = tqdm(enumerate(train_loader), total=total_it)
    for i, (inputs, labels) in prog_bar:
        
        counter += 1
        optimizer.zero_grad()
        inputs = inputs.to(device)
        labels = labels.to(device)
        total += labels.size(0)
        
        
        outputs = model(inputs)
#         print(labels.shape)
#         print(outputs.shape)
#         print(labels)
#         print(outputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        _, preds = torch.max(outputs.data, 1)
        train_running_correct += (preds == labels).sum().item()
        
        train_running_loss += loss.item()
        
        if i == total_it//2 or i == total_it-1:
            train_loss = train_running_loss / counter
            train_accuracy = 100. * train_running_correct / total

            print("Loss :", train_loss)
            print("Accuracy :", train_accuracy)
            print("VALIDATION")
            check_f1(val_loader, len(val_dataset), model, device)
            print("-"*50)
    train_running_loss = 0.0
    train_running_correct = 0
print("Finish")

  0%|          | 0/506 [00:00<?, ?it/s]

torch.Size([64])
torch.Size([64, 18])
tensor([15, 17,  5,  4,  4,  4, 17,  3, 15,  4,  6,  3,  0,  9,  9,  5,  4, 10,
         5,  9,  9, 13, 16,  9,  3, 14, 13, 16,  2, 10, 10, 10, 16,  4,  3, 12,
        12,  1,  0, 13,  9, 11, 15,  0, 10, 17, 17,  2,  7, 10,  9, 15, 14, 12,
        15, 15,  2,  5, 15, 17,  0,  3,  5,  6], device='cuda:0')
tensor([[ 0.0779,  0.2769, -0.3150,  ...,  0.0709, -0.0359,  0.1472],
        [-0.0267,  0.0856,  0.1728,  ..., -0.0639,  0.1057,  0.1735],
        [-0.2838,  0.0887, -0.0521,  ..., -0.0049, -0.2390, -0.2795],
        ...,
        [ 0.3627,  0.2630,  0.2807,  ...,  0.0240,  0.1097,  0.1130],
        [-0.3551,  0.2651, -0.0801,  ...,  0.0845,  0.0675,  0.2518],
        [ 0.1563, -0.0016, -0.1610,  ...,  0.0270, -0.1215, -0.1958]],
       device='cuda:0', grad_fn=<AddmmBackward>)


  0%|          | 1/506 [00:01<08:40,  1.03s/it]

torch.Size([64])
torch.Size([64, 18])
tensor([10, 17, 15,  0,  6, 14,  6,  9,  3, 11,  7,  4, 17,  3,  6,  4, 10,  1,
         3, 15,  9,  5,  4,  4, 14, 12,  6, 16, 10,  9,  6,  3,  3, 12,  4, 12,
         3, 16,  5, 15,  0, 13, 16,  0,  9, 11,  9, 15,  1, 11,  7, 16,  6, 16,
        11, 10,  7,  0,  3,  4,  8, 17, 16,  3], device='cuda:0')
tensor([[-0.0445, -0.1637, -0.0078,  ...,  0.1566,  0.1889, -0.0556],
        [-0.0301, -0.1406, -0.2427,  ...,  0.1745,  0.1009,  0.2529],
        [ 0.0921, -0.0415,  0.0473,  ...,  0.2824,  0.2231, -0.4373],
        ...,
        [ 0.0131,  0.0085,  0.0316,  ...,  0.3194,  0.1628,  0.0833],
        [ 0.2901,  0.2583,  0.6442,  ...,  0.2840, -0.2113,  0.1758],
        [-0.0070,  0.0687, -0.0085,  ..., -0.0344, -0.2115,  0.1626]],
       device='cuda:0', grad_fn=<AddmmBackward>)


  0%|          | 2/506 [00:01<07:10,  1.17it/s]

torch.Size([64])
torch.Size([64, 18])
tensor([10,  6, 10,  9, 12, 10,  4, 15,  7,  3, 12,  9,  9,  0,  9,  3,  0, 10,
         3,  0, 17,  3,  9,  1, 16,  6, 11, 10, 16, 14, 10, 10, 12,  2, 15, 16,
         4, 15, 10,  3, 13,  5, 10, 10, 15,  1,  1,  9, 11,  9,  9, 15, 16, 11,
        15,  3,  1, 15, 15, 12,  9, 12, 13,  3], device='cuda:0')
tensor([[ 0.1325,  0.1009, -0.2671,  ..., -0.1872,  0.0753, -0.2709],
        [ 0.2965,  0.1585, -0.0417,  ...,  0.4029, -0.1907, -0.0940],
        [ 0.2387, -0.0300, -0.0659,  ...,  0.1064, -0.0170, -0.1823],
        ...,
        [-0.2176, -0.1806,  0.0068,  ..., -0.0483,  0.0059, -0.0210],
        [-0.1708,  0.0567,  0.0477,  ..., -0.1530,  0.1756, -0.2642],
        [ 0.2571,  0.0623,  0.1388,  ...,  0.0446, -0.1363,  0.0916]],
       device='cuda:0', grad_fn=<AddmmBackward>)


  1%|          | 3/506 [00:02<05:59,  1.40it/s]

torch.Size([64])
torch.Size([64, 18])
tensor([12, 16,  1, 17, 10, 16, 16, 16,  0,  8, 12, 11, 13, 14, 13, 13, 10, 14,
        16,  3, 17,  7,  0,  0, 14,  3, 10, 15,  8, 10,  1,  0,  9,  3, 15, 17,
         9, 15,  9, 10,  9,  8, 10, 12, 11,  9, 16, 12, 16, 14, 12,  4,  3,  6,
         4,  5, 14,  7,  9,  5,  0, 14,  4,  0], device='cuda:0')
tensor([[-0.1385, -0.1266, -0.4030,  ...,  0.3999, -0.0439, -0.0337],
        [ 0.2078, -0.1077, -0.2657,  ...,  0.4572, -0.1859, -0.1695],
        [-0.0531, -0.0242, -0.0125,  ...,  0.0263, -0.0775, -0.0868],
        ...,
        [-0.2823,  0.3330, -0.3532,  ..., -0.0620,  0.2539,  0.1009],
        [-0.0494, -0.1030, -0.5128,  ..., -0.2821, -0.1620, -0.5189],
        [ 0.2025,  0.0563, -0.2505,  ...,  0.0499, -0.3958,  0.0450]],
       device='cuda:0', grad_fn=<AddmmBackward>)


KeyboardInterrupt: 

## Inference

In [ ]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
dataset = TestDataset(image_paths, tfms)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
# device = torch.device('cuda')

model.eval()
device = torch.device("cuda:0")
# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []

prog_bar = tqdm(enumerate(loader), total=int(len(dataset)/loader.batch_size))
for i, images in prog_bar:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

In [ ]:
len(dataset)